<a href="https://colab.research.google.com/github/eduartheinen/foursquare-tips/blob/master/foursquare-tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download pt_core_news_sm

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 1683, in _parseNoCache
    loc, tokens = self.parseImpl(instring, preloc, doActions)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 2891, in parseImpl
    if instring[loc] == self.firstMatchChar and instring.startswith(self.match, loc):
IndexError: string index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

In [2]:
import re
import string

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2, SelectKBest
from spacy.lang.pt import Portuguese
from spacy.lang.pt.stop_words import STOP_WORDS

In [15]:
class FoursquareTipsDataset:
    def __init__(self, df, ngram_range=None):
        self.reviews = [self.preprocess(s) for s in df.texto]
        self.labels = df.rotulo

        self.count_vectorizer = CountVectorizer(ngram_range=ngram_range,
                                                stop_words=STOP_WORDS, 
                                                strip_accents='unicode')
        self.bow = self.count_vectorizer.fit_transform(self.reviews)  # bag of words

        self.tfidf_vectorizer = TfidfVectorizer(ngram_range=ngram_range, stop_words=STOP_WORDS)
        self.tfidf = self.tfidf_vectorizer.fit_transform(self.reviews)

    @staticmethod
    def preprocess(text):
      if type(text) != str:
        print(f'houston {type(text)}')
      punctuation_to_space = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
      text = text.translate(punctuation_to_space)  # change punctuations to spaces
      text = re.sub('  +', ' ', text)  # removes double spaces
      text = re.sub('\d+', '', text)  # removes numbers
      text = str.lower(text)  # to lowercase
      # text = [unidecode(w) for w in text.split() if w not in STOP_WORDS]  # removes accent marks

      return text

    def __getitem__(self, i):
        return self.reviews[i], self.labels[i], self.tfidf[i]

    def __len__(self):
        return len(self.sentences)

In [16]:
path = 'https://raw.githubusercontent.com/eduartheinen/foursquare-tips/master/data/'
df = pd.read_csv(path + 'tips_scenario1_train.csv').dropna(how='any')

data = FoursquareTipsDataset(df, ngram_range=(1, 2)) 

['ai', 'alem', 'apos', 'area', 'ate', 'atras', 'atraves', 'ca', 'dao', 'devera', 'entao', 'es', 'estao', 'estara', 'faco', 'fara', 'ira', 'ja', 'la', 'maximo', 'mes', 'nao', 'nivel', 'numero', 'numeros', 'podera', 'poe', 'poem', 'porem', 'posicao', 'possivel', 'proprio', 'proxima', 'proximo', 'questao', 'relacao', 'sao', 'setima', 'setimo', 'so', 'tambem', 'tao', 'tres', 'ultimo', 'vao', 'varios', 'voce', 'voces']

# Repositório de Word-Embeddings em Português do NILC-ICMC-USP http://www.nilc.icmc.usp.br/embeddings
# embeddings_path = "http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s100.zip"

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ai', 'alem', 'apos', 'area', 'ate', 'atras', 'atraves', 'ca', 'dao', 'devera', 'entao', 'es', 'estao', 'estara', 'faco', 'fara', 'ira', 'ja', 'la', 'maximo', 'mes', 'nao', 'nivel', 'numero', 'numeros', 'podera', 'poe', 'poem', 'porem', 'posicao', 'possivel', 'proprio', 'proxima', 'proximo', 'questao', 'relacao', 'sao', 'setima', 'setimo', 'so', 'tambem', 'tao', 'tres', 'ultimo', 'vao', 'varios', 'voce', 'voces'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
ch2 = SelectKBest(chi2, k=20)
tmp = ch2.fit_transform(data.tfidf, data.labels)
features = data.tfidf_vectorizer.get_feature_names()
selected_features = [{features[i]: ch2.scores_[i]} for i in ch2.get_support(indices=True)]
selected_features

In [ ]:
ch2 = SelectKBest(chi2, k=20)
class_indexes = data.labels[data.labels == 1]
# tmp = ch2.fit_transform(data.reviews[class_indexes])
# selected_features = [{features[i]: ch2.scores_[i]} for i in ch2.get_support(indices=True)]
# selected_features